In [19]:
# 区間推定

# 「区間推定」とは、データを用いて未知母数Θの値の存在範囲を区間として推定する方法である
# たとえば、標準正規分布(N(0,1))の上側2.5%点、および上側97.5%点から以下の確率式が成り立つ
# P(-1.96<=μ<=1.96) = 0.95
# この時の確率0.95は「信頼率」あるいは「信頼係数」とよび、標本に依存して得られる区間を「信頼区間」、
# 信頼区間の上限と下限を「信頼限界」とよぶ
# 信頼率を大きくすると、信頼区間がμを含む確率は大きくなるが、区間幅が大きくなりμの値を推測する上での有用性を失う
# 信頼率を一定に保ったまま区間幅を大きくするには、標本サイズnを大きくすればよい

import math
import pandas as pd
from scipy.stats import norm
from sklearn.datasets import load_iris

iris = load_iris()
iris_df = pd.DataFrame(
        iris.data,
        columns=iris.feature_names,
    )

sl_df_num =100
sl_df = iris_df["sepal length (cm)"]
sl_sample_df = sl_df.sample(n=sl_df_num) # 100このデータをサンプルとして使用

p_var = sl_df.var() # 母分散
s_mean = sl_sample_df.mean() # 標本平均


bottom, up = norm.interval(0.95, loc=s_mean, scale=math.sqrt(p_var/sl_df_num))
print(f"母平均μの信頼区間：{bottom}<=μ<={up}")
print(f"母平均μ:{sl_df.mean()}")

母平均μの信頼区間：5.654702021234584<=μ<=5.979297978765414
母平均μ:5.843333333333334


In [21]:
# 分散の区間推定

# 母集団からの独立な標本X1, X2,.., Xnの標本平均X'からの偏差平方和をT**2とすると
# χ**2=T**2/σ**2は自由度n-1のカイ二乗分布に従う

import numpy as np
from scipy.stats import chi2

sigma_hat2 = np.var(sl_sample_df, ddof=1)

chi2_lower, chi2_upper = chi2.interval(0.95, df=sl_df_num-1)
print(f"母分散の信頼区間：{sl_df_num*sigma_hat2/chi2_lower}<=σ**2<={sl_df_num*sigma_hat2/chi2_upper}")
print(f"母分散:{sl_df.var()}")


母分散の信頼区間：1.130992180928467<=σ**2<=0.6460794522573198
母分散:0.6856935123042507


In [27]:
# 分散の比の区間推定

# 2つの独立した集団における分散を比較するための指標として分散の比が用いられる
# それぞれの集団が独立に正規分布に従うとき、統計量Fは
# F = (V1/σ1**2)/(V2/σ2**2)は、自由度(n1-1, n2-1)のF分布に従う
# P(V1/V2*1/F0.025 <= σ1**2/σ2**2 <= V1/V2*1/F0.975) = 0.95 となる

from scipy.stats import f

sw_df = iris_df["sepal width (cm)"]
sw_df_num = 100
sw_sample_df = sw_df.sample(n=sw_df_num) # 100このデータをサンプルとして使用

sl_sigma_hat2 = np.var(sl_sample_df, ddof=1)
sw_sigma_hat2 = np.var(sw_sample_df, ddof=1)

f_lower, f_upper = f.interval(0.95, dfn=sl_df_num-1, dfd=sw_df_num-1)
print(f"母比率の信頼区間：{sl_sigma_hat2/sw_sigma_hat2*1/f_lower}<=σ1**2/σ2**2<={sl_sigma_hat2/sw_sigma_hat2*1/f_upper}")
print(f"母比率:{sl_df.var()/sw_df.var()}")

母比率の信頼区間：6.059837313269355<=σ1**2/σ2**2<=2.7433847251825494
母比率:3.609304198735764


In [42]:
# 多項分布の信頼区間

# 確率piで事象Ai(i=1,2,..,k)が起こるような試行をn回行ったとき、
# それぞれの事象が起こる回数Niに関する確率分布を多項分布という
# ある事象Aiが起こるかどうかに着目すると、多項分布は二項分布に帰着するため、
# E[Ni]=n*pi, V[Ni]=n*pi*(1-pi)となる
# ここでpi'=Ni/nとし、ui=(pi'-pi)/(√pi*(1-pi))/n)とすると、
# uiは漸近的に標準正規分布に従いP(-1.96<=ui<=1.96)=0.95が成り立つ

# サイコロの例
import random

dice = list(range(1,7))
dice_selected = 1
n_all = 1000
n1 = random.choices(dice, k=n_all).count(dice_selected)

p1_hat = n1/n_all
ui_denominator = math.sqrt(p1_hat*(1-p1_hat)/n_all)

multi1_lower, multi1_upper = norm.interval(0.95, loc=0, scale=1)
print(f"pi(サイコロ1)の信頼区間：{p1_hat+multi1_lower*ui_denominator:.3f}<=pi<={p1_hat+multi1_upper*ui_denominator:.3f}")
print(f"pi(サイコロ1):{1/6:.3f}")


pi(サイコロ1)の信頼区間：0.138<=pi<=0.184
pi(サイコロ1):0.167


In [48]:
# 多項分布の差の信頼区間

# p1-p2の信頼区間を求めるために、p1'-p2'=N1/n-N2/nの期待値と分散を考える
# p1'-p2'の期待値はE[p1'-p2']=p1-p2, 
# p1'-p2'の分散はV[p1'-p2']=p1*(1-p1)/n + p2*(1-p2)/n + 2*p1*p2/nとなる
# u = (p1'-p2'-E[p1'-p2'])/√[p1'-p2']とすると、uは漸近的に標準正規分布に従い、
# p1-p2の95%信頼区間は、p1'-p2'-1.96*√V[p1'-p2'], p1'-p2'+1.96*√V[p1'-p2']となる

dice_selected = 2
n2 = random.choices(dice, k=n_all).count(dice_selected)
p2_hat = n2/n_all

u_denominator = math.sqrt(
    p1_hat*(1-p1_hat)/n_all+\
    p2_hat*(1-p2_hat)/n_all+\
    2*p1_hat*p2_hat/n_all
)

print(f"p1-p2の信頼区間：{(p1_hat-p2_hat)+multi1_lower*u_denominator:.3f}<=p1-p2<={(p1_hat-p2_hat)+multi1_upper*u_denominator:.3f}")
print(f"p1-p2:{1/6-1/6:.3f}")


p1-p2の信頼区間：-0.023<=p1-p2<=0.047
p1-p2:0.000
